# Parse an IPAC table to json for d3

*I will want to clean this up later when I decide the exact format for the interactive*

In [1]:
from astropy.table import Table
import numpy as np
import csv
import pandas as pd

from astropy import units as u
from matplotlib import pyplot as plt
from astropy.stats import LombScargle
from scipy.signal import argrelextrema

In [2]:
filename = "27882110006813.ipac_tbl"
ipac_lc = Table.read(filename, format='ipac')

hjd = np.array(ipac_lc["obsmjd"])
mag = np.array(ipac_lc["mag_autocorr"])
mag_unc = np.array(ipac_lc["magerr_auto"])

mmag = np.mean(mag) - mag

### Create the periodogram output file

In [3]:
# Note: these are all in units of days
############
# periodogram and best-fit period
pmin = 0.1
pmax = 50.
ls = LombScargle(hjd, mag)
frequency, power = ls.autopower(maximum_frequency=1./pmin, minimum_frequency=1./pmax)
best_frequency = frequency[np.argmax(power)]
period = 1./frequency
best_period = 1./best_frequency
print("best period", best_period)


best period 0.4661477095501411


### Create some dummy data for the 2nd filter

In [4]:
Nobs = len(hjd)
pfac = 2.2
hjd2 = np.random.random(size=Nobs)*max(hjd - np.min(hjd))
mmag2 = np.sin(2.*np.pi/(pfac*best_period) * hjd2)
mag_unc2 = mag_unc

### Write everything to a single json file

*Note: the default in the interactive will be to use the first filter's period for phasing when page loads.*

In [5]:
def dataToJson(filterNames, colors, CMDdata, LCdata, periods,
               multiples=[1,0.5, 2., 3.], 
               multiplesNames=["whole","half","twice","triple"], 
               features = None, 
               featureRange = [[0.1, 1000], [0.01, 3]],
               featureFormat = ['log', 'log'],
               filename="LCdata.json")  :
    """
    #filter names (not visible in interactive, but should be unique strings)
    filterNames = ['filt1', 'filt2']

    #colors for data in each filter
    colors = ['#dc143c','#4682b4']
    
    #CMDdata should be a dict with the following keys:
    CMDdata = {
     "color" : #the mag1 - mag2 color of the object
     "mag" : #the mag1 or mag2 value
     "magP" : #a list of percentile values for the magnitude (e.g., 90, 95, 99 confidence intervals)
     "colorP" : #a list of percentile values for the color (same confidence intervals as magP)
     "cs" : #the colors for each percentile
     }
     
    #LCdata should be a dict of dicts, one for each filter with the following keys:
    LCdata[filter] = {
        "hjd" : # a list of the hjd dates of observations
        "mag" : # a list of the magnitudes in the given "filter"
        "err" : # a list of the uncertainties on the magnitues in the given "filter 
        "r": # optional, the size of the circle for plotting (single number); default is 3
     }
     
    #periods for each filter (in same order as filterNames)
    periods = [1.2345,1.2346]
    
    ##########
    #the following inputs are optional
    ###########
    
    #multiplicative factor for the period
    multiples = [1, 0.5, 2., 3.] 
    
    #names for the buttons associated with each multiple
    multiplesNames=["whole period","half the period","twice the period","triple the period"] 

    #any data that should appear in the features spines; 
    #must include "period" first!, and ordered in the same way as filterNames
    #if "features" are not defined on input, the following features are calculated
    features = {'period':[best_period, pfac*best_period],
            'amplitude':[max(mmag) - min(mmag), max(mmag2) - min(mmag2)]}
    
    #range for axes 
    featureRange = [[0.1, 1000], [0.01, 3]] 
    
    #log vs. linear for feature spines
    featureFormat = ['log', 'log']  

    #name of the output file
    filename = "LCdata.json"
    """
    
    #create the output dict, and begin defining values
    outDict = {}
                             
    #create LC data 
    outDict['rawData'] = []
    if (features == None):
        amplitudes = []
    for i,f in enumerate(filterNames):
        r = 3.
        if "r" in LCdata[f]:
            r = LCdata[f]["r"]
        if (features == None):
            amplitudes.append(max(LCdata[f]['mag']) - min(LCdata[f]['mag']))
            
        for x,y,ye in zip(LCdata[f]['hjd'], LCdata[f]['mag'], LCdata[f]['err']):
            data = {
                "x":x - min(LCdata[f]['hjd']),
                "y":y,
                "ye":ye,
                "r":r,
                "c":colors[i],
                "filter":filterNames[i]
            }
            outDict['rawData'].append(data)
        
    #create the CMD data
    outDict['CMDdata'] = []
    for rx,ry,c in zip(CMDdata['colorP'], CMDdata['magP'], CMDdata['cs']):
        data={
            "x":CMDdata['color'], 
            "y":CMDdata['mag'],
            "rx":rx,
            "ry":ry, 
            "c":c}
        outDict['CMDdata'].append(data)

    #feature data
    if (features == None):
        features = {'period':periods, 'amplitude':amplitudes}
               
    outDict['features'] = list(features.keys())
    outDict['featuresRange'] = featureRange 
    outDict['featuresFormat'] = featureFormat 
    outDict['featureData'] = {}
    for f in list(features.keys()):
        outDict['featureData'][f] = []
        for i,val in enumerate(features[f]):
            outDict['featureData'][f].append({
                "x":0,
                "y":val,
                "c":colors[i],
                "filter":filterNames[i]
            })


    #some additional items 
    outDict['filters'] = filterNames
    outDict['multiples'] = multiples
    outDict['multiplesNames'] = multiplesNames
    for i,f in enumerate(filterNames):
        outDict[f] = {}
        outDict[f]['color'] = colors[i]
        outDict[f]['period'] = periods[i]

    
    pd.Series(outDict).to_json(filename, orient='index')

In [6]:
#filter names (not visible in interactive, but should be unique strings)
filterNames = ['filt1', 'filt2']

#colors for data in each filter
colors = ['#dc143c','#4682b4']

#best fit periods for each filter (in the same order as filterNames)
periods = [best_period, pfac*best_period]

#CMD data (dummy data for now)
#percentiles listed from large to small
CMDdata = dict()
CMDdata['color'] = 2
CMDdata['mag'] = 5
CMDdata['magP'] = [15,10,3]
CMDdata['colorP'] = [10,5,3]
CMDdata['cs'] = ["#470713","#dc143c","#ffb5c2"]

#light curve data, combined into dict
LCdata = dict()
LCdata[filterNames[0]] = dict()
LCdata[filterNames[0]]['hjd'] = hjd
LCdata[filterNames[0]]['mag'] = mmag
LCdata[filterNames[0]]['err'] = mag_unc
LCdata[filterNames[1]] = dict()
LCdata[filterNames[1]]['hjd'] = hjd2
LCdata[filterNames[1]]['mag'] = mmag2
LCdata[filterNames[1]]['err'] = mag_unc2

#create the output json
dataToJson(filterNames, colors, CMDdata, LCdata, periods, filename="27882110006813.json")



### In case these are needed later...

*Write to a csv file*

In [ ]:
csvfile = open("27882110006813.csv", 'wt')
csvwriter = csv.writer(csvfile)
csvwriter.writerow(["hjd","mag","emag"])
for i in range(len(hjd)):
    csvwriter.writerow([hjd[i], mmag[i], mag_unc[i]])
csvfile.close()

In [ ]:
csvfile = open("27882110006813_periods.csv", 'wt')
csvwriter = csv.writer(csvfile)
csvwriter.writerow(["period"])
for i in range(len(ptest_final)):
    csvwriter.writerow([ptest_final[i]])
csvfile.close()

In [ ]:
#some additional periods that might be of interest

############
# Get the harmonics f/2, f/3
htests = np.array([best_frequency, best_frequency/2., best_frequency/3.])

############
# Get a few common failures : frequency +/- integers (for a typical 1 day observing cadence) 
ftests = np.array([])
rng = np.array([1, 2])
for i in rng:
    hm = [htests + i, htests - i]
    ftests = np.append(ftests, hm)
ftests = np.unique(ftests) #also sorts
useit = np.where(ftests > 0)
ftests = ftests[useit]

############
#Simply get some of the top maxima, but exclude any that have a high false-alarm probability
spacing = 50 # number of neighboring points to consider for finding relative maxima
alarm = 1e-4 # maximum of false alarm probability to accept
posall = argrelextrema(power, np.greater, order=spacing)[0] #get all maxima
fx = frequency[posall]
powx = power[posall]

# test the false-alarm probability
falarm = ls.false_alarm_probability(powx)
useit = np.where(falarm < alarm)
ftests2 = fx[useit]


############
#Sort these in order of importance for the user.  
#I will keep the best fit and harmonics first, then append a combined list of these last two sorted by power
#Combine the common failures and extra maxima
ftAll = np.append(ftests, ftests2)
ptAll = np.interp(ftAll, frequency, power)
ft, fi = np.unique(ftAll, return_index=True)
pt = ptAll[fi]
sp = np.argsort(pt)
sp = sp[::-1]
ft = ft[sp]
# remove the harmonics from this list
ft1 = np.array([])
for f in ft:
    test = np.where(htests == f)[0]
    if (len(test) == 0):
        ft1 = np.append(ft1, f)
        
# now prepend the harmonics 
ftest_final = np.append(htests, ft1)
ptest_final = 1/ftest_final

print(ptest_final)